# Data Preparation

In [24]:
import pandas as pd

#### Original Dataset

In [25]:
df=pd.read_csv("retail_transactions.csv")

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,546729,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2011-03-16 11:36:00,1.25,18231.0,United Kingdom
1,559898,21868,POTTING SHED TEA MUG,6,2011-07-13 12:18:00,1.25,16225.0,United Kingdom
2,548648,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-04-01 13:20:00,0.85,12949.0,United Kingdom
3,540543,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,4,2011-01-09 15:23:00,2.95,14395.0,United Kingdom
4,561390,20726,LUNCH BAG WOODLAND,10,2011-07-27 09:52:00,1.65,17068.0,United Kingdom


##### Output dataset. Apply data manipulation technique

In [26]:
processed=pd.read_csv("wrangled_transactions.csv")

processed.head()

,CustomerID,2010 revenue,days_since_first_purchase,days_since_last_purchase,number_of_purchases,avg_order_cost,2011 revenue
0,12347.0,711.79,23.0,23.0,1.0,711.79,3598.21
1,12348.0,892.80,14.0,14.0,1.0,892.80,904.44
2,12370.0,1868.02,16.0,13.0,2.0,934.01,1677.67
3,12377.0,1001.52,10.0,10.0,1.0,1001.52,626.60
4,12383.0,600.72,8.0,8.0,1.0,600.72,1249.84


## Data pre-processing

In [27]:
df=df[["Quantity","InvoiceDate","UnitPrice","CustomerID"]]

df.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID
0,12,2011-03-16 11:36:00,1.25,18231.0
1,6,2011-07-13 12:18:00,1.25,16225.0
2,24,2011-04-01 13:20:00,0.85,12949.0
3,4,2011-01-09 15:23:00,2.95,14395.0
4,10,2011-07-27 09:52:00,1.65,17068.0


In [28]:
# Calculate the revenue 
df["revenue"]=df.Quantity*df.UnitPrice

df.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,revenue
0,12,2011-03-16 11:36:00,1.25,18231.0,15.0
1,6,2011-07-13 12:18:00,1.25,16225.0,7.5
2,24,2011-04-01 13:20:00,0.85,12949.0,20.4
3,4,2011-01-09 15:23:00,2.95,14395.0,11.8
4,10,2011-07-27 09:52:00,1.65,17068.0,16.5


In [29]:
# Convert datetime column to datetime object

df["InvoiceDate"]=pd.to_datetime(df.InvoiceDate)

df.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,revenue
0,12,2011-03-16 11:36:00,1.25,18231.0,15.0
1,6,2011-07-13 12:18:00,1.25,16225.0,7.5
2,24,2011-04-01 13:20:00,0.85,12949.0,20.4
3,4,2011-01-09 15:23:00,2.95,14395.0,11.8
4,10,2011-07-27 09:52:00,1.65,17068.0,16.5


In [42]:
# Adding a column Year
import datetime

df["Year"]= [datetime.datetime.strftime(day, "%Y-%m-%d").split("-")[0] for day in df.InvoiceDate]

df.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,revenue,Year
0,12,2011-03-16 11:36:00,1.25,18231.0,15.0,2011
1,6,2011-07-13 12:18:00,1.25,16225.0,7.5,2011
2,24,2011-04-01 13:20:00,0.85,12949.0,20.4,2011
3,4,2011-01-09 15:23:00,2.95,14395.0,11.8,2011
4,10,2011-07-27 09:52:00,1.65,17068.0,16.5,2011


In [43]:
# CostomerID should be integer
df["CustomerID"]=[int(ID) for ID in df.CustomerID]

df.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,revenue,Year
0,12,2011-03-16 11:36:00,1.25,18231,15.0,2011
1,6,2011-07-13 12:18:00,1.25,16225,7.5,2011
2,24,2011-04-01 13:20:00,0.85,12949,20.4,2011
3,4,2011-01-09 15:23:00,2.95,14395,11.8,2011
4,10,2011-07-27 09:52:00,1.65,17068,16.5,2011


In [162]:
# Produce the same data as requested in Output section
ID=set(df.CustomerID)
yearSet=set(df.Year)

newDict={"CustomerID":[],"firstDay":[], "lastDay":[],"buyCount":[],"meanCost":[],\
        "2010":[],"2011":[]}

for eachID in ID:
    sub_ID=df[df.CustomerID==eachID] 
    sortDate=sub_ID.sort_values("InvoiceDate")
    sortDate.reset_index(drop=True, inplace=True)
    newDict["firstDay"].append(sortDate.InvoiceDate.dt.strftime("%Y-%m-%d").values.tolist()[0].split("-")[-1])
    newDict["lastDay"].append(sortDate.InvoiceDate.dt.strftime("%Y-%m-%d").values.tolist()[-1].split("-")[-1])
    newDict["CustomerID"].append(eachID)
    newDict["buyCount"].append(sortDate.Quantity.sum())
    newDict["meanCost"].append(sortDate.revenue.sum()/len(sortDate))
    year10=[]
    year11=[]
    for eachYear in yearSet:
        sub_Year=sub_ID[sub_ID.Year==eachYear]
        doanhThu=sum(sub_Year.revenue)
        if eachYear=="2010":
            year10.append(doanhThu)
        elif eachYear=="2011":
            year11.append(doanhThu)
    newDict["2010"].append(year10[-1])
    newDict["2011"].append(year11[-1])        

In [169]:
df_processed=pd.DataFrame(newDict)

df_Final=df_processed.rename(columns={"firstDay":"days_since_first_purchase","lastDay":"days_since_last_purchase","buyCount":"number_of_purchases",\
                                     "meanCost":"avg_order_cost","2010":"2010_revenue","2011":"2011_revenue"})
df_Final.head()

,CustomerID,days_since_first_purchase,days_since_last_purchase,number_of_purchases,avg_order_cost,2010_revenue,2011_revenue
0,16384,21,11,260,17.712121,0.00,584.50
1,16385,08,10,261,9.421864,148.23,407.66
2,16386,31,11,224,3.604545,0.00,317.20
3,16387,21,21,44,23.590000,0.00,94.36
4,16389,01,17,1242,40.649118,0.00,1382.07
